In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import numpy as np

In [3]:
import sys
sys.path.insert(0, "/home/storage/hans/jax_reco/python")
from geo import closest_distance_dom_track, closest_distance_dom_track_v
from geo import light_travel_time, light_travel_time_v
from geo import z_component_closest_point_on_track, z_component_closest_point_on_track
from geo import light_travel_time_i3calculator, light_travel_time_i3calculator_v
from geo import convert_spherical_to_cartesian_direction
from geo import cherenkov_cylinder_coordinates, cherenkov_cylinder_coordinates_v

In [4]:
from icecube import dataclasses
from icecube.phys_services import I3Calculator
# can use the following functions:
#    I3Calculator.cherenkov_time
#    I3Calculator.closest_approach_distance
#    I3Calculator.closest_approach_position
# example: double I3Calculator::CherenkovTime(const I3Particle& particle, 
# const I3Position& position, const double IndexRefG, const double IndexRefP )

from icecube.icetray import I3Units

In [6]:
# from icetray
# https://github.com/icecube/icetray/blob/main/dataclasses/public/dataclasses/I3Constants.h
#__n_ice_phase = 1.3195;
#__n_ice_group = 1.35634;
#__n_ice = __n_ice_group

# or use the values from clsim in our tables
#  'n_group': 1.32548384613875,
#  'n_phase': 1.30799291638281
__n_ice_phase_clsim = 1.30799291638281
__n_ice_group_clsim = 1.32548384613875
__n_ice_clsim = __n_ice_group_clsim

__n_ice_phase = __n_ice_phase_clsim
__n_ice_group = __n_ice_group_clsim
__n_ice = __n_ice_clsim

__theta_cherenkov = np.arccos(1/__n_ice_phase)
__c = 0.299792458 # m / ns
__c_ice = __c/__n_ice_group

# from Matti's code
# https://github.com/HansN87/gamma_mixture_photonics/blob/reconstruction/evaluation%20scripts/GetLLHs.py

_recip__speedOfLight = 3.3356409519815204
#_n__ = 1.32548384613875
#_tan__thetaC = (_n__**2.-1.)**0.5

# some comparisons point to difference in n_phase:
#print(np.tan(__theta_cherenkov), _tan__thetaC)
# 0.8608601802848123 0.8700042680210099

#print(__n_ice_phase, _n__)
# 1.3195 1.32548384613875

#print((__n_ice_phase**2.-1.)**0.5)
# 0.8608601802848125

In [7]:
# the icetray way ... ugh. ever heard the phrase "something being non-pythonic" ....
# will use these as point of comparison.

def get_icecube_track_i3particle(track_pos, track_dir):
    # track_pos: 3 component cartesian vector
    # track_dir: 2 component spherical vector (theta, phi)
    pos = dataclasses.I3Position(track_pos[0], track_pos[1], track_pos[2])
    d = dataclasses.I3Direction()
    d.set_theta_phi(track_dir[0], track_dir[1])
    pt = 0.0 # set vertex time to 0
    i3p = dataclasses.I3Particle(pos, d, pt, dataclasses.I3Particle.ParticleShape.InfiniteTrack, 100000 * I3Units.m)
    return i3p

def get_cherenkov_time(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.cherenkov_time(p, pos, __n_ice_group, __n_ice_phase)

def get_closest_approach_distance(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_distance(p, pos)

def get_closest_approach_position(p, dom_pos):
    pos = dataclasses.I3Position(dom_pos[0] * I3Units.m, dom_pos[1] * I3Units.m, dom_pos[2] * I3Units.m)
    return I3Calculator.closest_approach_position(p, pos)

In [8]:
def light_travel_time_eijndhoven(dom_pos, track_pos, track_dir):
    """
    eq. 3 / Fig. 1 of https://arxiv.org/pdf/0704.1706
    """
    v_a = dom_pos - track_pos

    closest_dist = closest_distance_dom_track(dom_pos, track_pos, track_dir)
    d1 = jnp.dot(v_a, track_dir)
    
    dt = _recip__speedOfLight * (d1 + closest_dist * (__n_ice_group_clsim * __n_ice_phase_clsim - 1) / jnp.sqrt(__n_ice_phase_clsim * __n_ice_phase_clsim -1))
    return dt

In [9]:
# single example:

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)
dom_pos = np.array([0, 0, -150])

print(get_cherenkov_time(i3track, dom_pos)) # geo time
print(get_closest_approach_distance(i3track, dom_pos)) # closest distance to dom
print(get_closest_approach_position(i3track, dom_pos)[2]) # z component of closest position on track

CUDA backend failed to initialize: Unable to use CUDA because of the following issues with CUDA components:
Outdated cuDNN installation found.
Version JAX was built against: 8906
Minimum supported: 8900
Installed version: 8101
The local installation version must be no lower than 8900. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


199.95201012714827
10.375040484114887
-149.85830716519607


In [10]:
print(light_travel_time_i3calculator(dom_pos, track_pos, track_dir))

199.95201012714833


In [11]:
dt, s, z = cherenkov_cylinder_coordinates(dom_pos, track_pos, track_dir)
print(dt)
print(s)
print(z)

199.95201012714833
10.375040484114885
-149.85830716519607


In [12]:
print(light_travel_time(dom_pos, track_pos, track_dir)) # ~1.5 ns difference compared to icetray's I3Calculator.
print(closest_distance_dom_track(dom_pos, track_pos, track_dir))
print(z_component_closest_point_on_track(dom_pos, track_pos, track_dir))

199.94345145110827
10.375040484114885
-149.85830716519607


In [13]:
print(light_travel_time_eijndhoven(dom_pos, track_pos, track_dir))

199.95201012714827


In [ ]:
# now demonstrate a vectorized example

track_pos = np.array([-10., 10., -200.])
track_theta = np.deg2rad(10.)
track_phi = np.deg2rad(2.)
track_dir_sph = np.array([track_theta, track_phi])
track_dir = np.array(convert_spherical_to_cartesian_direction(jnp.array(track_dir_sph)))

i3track = get_icecube_track_i3particle(track_pos, track_dir_sph)

n_doms = 50
dom_pos = np.random.normal(0.0, 300, (n_doms, 3))

# single call that computes everything over the n_doms matrix
dt, s, z = cherenkov_cylinder_coordinates_v(dom_pos, track_pos, track_dir)

In [ ]:
# and the awkward icetray style loop
for i, d_pos in enumerate(dom_pos):
    dt_i3c = get_cherenkov_time(i3track, d_pos)
    print(f"delay time: {dt[i]}, {dt_i3c}") 

In [ ]:
for i, d_pos in enumerate(dom_pos):
    s_i3c = get_closest_approach_distance(i3track, d_pos)
    print(f"closest approach distance: {s[i]}, {s_i3c}") 

In [ ]:
for i, d_pos in enumerate(dom_pos):
    z_i3c = get_closest_approach_position(i3track, d_pos)[2]
    print(f"closest approach point (z-component): {z[i]}, {z_i3c}") 

